<a href="https://colab.research.google.com/github/dadoktor8/RAGProject-ML/blob/main/RAG_BTO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets pandas pymongo sentence_transformers
!pip install -U transformers


In [ ]:
from datasets import load_dataset
import pandas as pd

dataset = load_dataset('AIatMongoDB/embedded_movies')
dataset_df = pd.DataFrame(dataset['train']) #Assumption is that dataset has a collumn called train
dataset_df.head(5)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


sample_mflix.embedded_movies.json:  74%|#######4  | 31.5M/42.3M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

,plot,genres,runtime,cast,num_mflix_comments,poster,title,fullplot,languages,directors,writers,awards,imdb,countries,type,plot_embedding,rated,metacritic
0,Young Pauline is left a lot of money when her ...,[Action],199.0,"[Pearl White, Crane Wilbur, Paul Panzer, Edwar...",0,https://m.media-amazon.com/images/M/MV5BMzgxOD...,The Perils of Pauline,Young Pauline is left a lot of money when her ...,[English],"[Louis J. Gasnier, Donald MacKenzie]","[Charles W. Goddard (screenplay), Basil Dickey...","{'nominations': 0, 'text': '1 win.', 'wins': 1}","{'id': 4465, 'rating': 7.6, 'votes': 744}",[USA],movie,"[0.0007293965299999999, -0.026834568000000003,...",None,NaN
1,A penniless young man tries to save an heiress...,"[Comedy, Short, Action]",22.0,"[Harold Lloyd, Mildred Davis, 'Snub' Pollard, ...",0,https://m.media-amazon.com/images/M/MV5BNzE1OW...,From Hand to Mouth,As a penniless man worries about how he will m...,[English],"[Alfred J. Goulding, Hal Roach]",[H.M. Walker (titles)],"{'nominations': 1, 'text': '1 nomination.', 'w...","{'id': 10146, 'rating': 7.0, 'votes': 639}",[USA],movie,"[-0.022837115, -0.022941574000000003, 0.014937...",TV-G,NaN
2,"Michael ""Beau"" Geste leaves England in disgrac...","[Action, Adventure, Drama]",101.0,"[Ronald Colman, Neil Hamilton, Ralph Forbes, A...",0,None,Beau Geste,"Michael ""Beau"" Geste leaves England in disgrac...",[English],[Herbert Brenon],"[Herbert Brenon (adaptation), John Russell (ad...","{'nominations': 0, 'text': '1 win.', 'wins': 1}","{'id': 16634, 'rating': 6.9, 'votes': 222}",[USA],movie,"[0.00023330492999999998, -0.028511643000000003...",None,NaN
3,"Seeking revenge, an athletic young man joins t...","[Adventure, Action]",88.0,"[Billie Dove, Tempe Pigott, Donald Crisp, Sam ...",1,https://m.media-amazon.com/images/M/MV5BMzU0ND...,The Black Pirate,A nobleman vows to avenge the death of his fat...,None,[Albert Parker],"[Douglas Fairbanks (story), Jack Cunningham (a...","{'nominations': 0, 'text': '1 win.', 'wins': 1}","{'id': 16654, 'rating': 7.2, 'votes': 1146}",[USA],movie,"[-0.005927917, -0.033394486, 0.0015323418, -0....",None,NaN
4,An irresponsible young millionaire changes his...,"[Action, Comedy, Romance]",58.0,"[Harold Lloyd, Jobyna Ralston, Noah Young, Jim...",0,https://m.media-amazon.com/images/M/MV5BMTcxMT...,For Heaven's Sake,"The Uptown Boy, J. Harold Manners (Lloyd) is a...",[English],[Sam Taylor],"[Ted Wilde (story), John Grey (story), Clyde B...","{'nominations': 1, 'text': '1 nomination.', 'w...","{'id': 16895, 'rating': 7.6, 'votes': 918}",[USA],movie,"[-0.0059373598, -0.026604708, -0.0070914757000...",PASSED,NaN


In [ ]:
dataset_df = dataset_df.dropna(subset=["fullplot"])
print(dataset_df.isnull().sum())

plot                    0
genres                  0
runtime                14
cast                    1
num_mflix_comments      0
poster                 78
title                   0
fullplot                0
languages               1
directors              12
writers                13
awards                  0
imdb                    0
countries               0
type                    0
plot_embedding          1
rated                 279
metacritic            893
dtype: int64


In [ ]:
dataset_df = dataset_df.drop(columns=["plot_embedding"])
dataset_df.head(5)

,plot,genres,runtime,cast,num_mflix_comments,poster,title,fullplot,languages,directors,writers,awards,imdb,countries,type,rated,metacritic
0,Young Pauline is left a lot of money when her ...,[Action],199.0,"[Pearl White, Crane Wilbur, Paul Panzer, Edwar...",0,https://m.media-amazon.com/images/M/MV5BMzgxOD...,The Perils of Pauline,Young Pauline is left a lot of money when her ...,[English],"[Louis J. Gasnier, Donald MacKenzie]","[Charles W. Goddard (screenplay), Basil Dickey...","{'nominations': 0, 'text': '1 win.', 'wins': 1}","{'id': 4465, 'rating': 7.6, 'votes': 744}",[USA],movie,None,NaN
1,A penniless young man tries to save an heiress...,"[Comedy, Short, Action]",22.0,"[Harold Lloyd, Mildred Davis, 'Snub' Pollard, ...",0,https://m.media-amazon.com/images/M/MV5BNzE1OW...,From Hand to Mouth,As a penniless man worries about how he will m...,[English],"[Alfred J. Goulding, Hal Roach]",[H.M. Walker (titles)],"{'nominations': 1, 'text': '1 nomination.', 'w...","{'id': 10146, 'rating': 7.0, 'votes': 639}",[USA],movie,TV-G,NaN
2,"Michael ""Beau"" Geste leaves England in disgrac...","[Action, Adventure, Drama]",101.0,"[Ronald Colman, Neil Hamilton, Ralph Forbes, A...",0,None,Beau Geste,"Michael ""Beau"" Geste leaves England in disgrac...",[English],[Herbert Brenon],"[Herbert Brenon (adaptation), John Russell (ad...","{'nominations': 0, 'text': '1 win.', 'wins': 1}","{'id': 16634, 'rating': 6.9, 'votes': 222}",[USA],movie,None,NaN
3,"Seeking revenge, an athletic young man joins t...","[Adventure, Action]",88.0,"[Billie Dove, Tempe Pigott, Donald Crisp, Sam ...",1,https://m.media-amazon.com/images/M/MV5BMzU0ND...,The Black Pirate,A nobleman vows to avenge the death of his fat...,None,[Albert Parker],"[Douglas Fairbanks (story), Jack Cunningham (a...","{'nominations': 0, 'text': '1 win.', 'wins': 1}","{'id': 16654, 'rating': 7.2, 'votes': 1146}",[USA],movie,None,NaN
4,An irresponsible young millionaire changes his...,"[Action, Comedy, Romance]",58.0,"[Harold Lloyd, Jobyna Ralston, Noah Young, Jim...",0,https://m.media-amazon.com/images/M/MV5BMTcxMT...,For Heaven's Sake,"The Uptown Boy, J. Harold Manners (Lloyd) is a...",[English],[Sam Taylor],"[Ted Wilde (story), John Grey (story), Clyde B...","{'nominations': 1, 'text': '1 nomination.', 'w...","{'id': 16895, 'rating': 7.6, 'votes': 918}",[USA],movie,PASSED,NaN


In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('thenlper/gte-large')
def get_embedding(text: str)-> list[float]: #What is happening here?!
  if not text.strip():
    print('Text empty returned 0')
    return []
  embedding = model.encode(text) #Is this the part where the encoding is done? Probably!
  return embedding.tolist()

dataset_df["embedding"] = dataset_df["fullplot"].apply(get_embedding)
dataset_df.head

<bound method NDFrame.head of                                                    plot  \
0     Young Pauline is left a lot of money when her ...   
1     A penniless young man tries to save an heiress...   
2     Michael "Beau" Geste leaves England in disgrac...   
3     Seeking revenge, an athletic young man joins t...   
4     An irresponsible young millionaire changes his...   
...                                                 ...   
1495  In the ironically named city of Paradise, a re...   
1496  A group of suburban biker wannabes looking for...   
1497  Shakespeare's masterpiece "Othello" set in mod...   
1498  When a small Colorado town is overrun by the f...   
1499  A teenager with teleportation abilities sudden...   

                           genres  runtime  \
0                        [Action]    199.0   
1         [Comedy, Short, Action]     22.0   
2      [Action, Adventure, Drama]    101.0   
3             [Adventure, Action]     88.0   
4       [Action, Comedy, Romance]     58.0   
...                           ...      ...   
1495   [Action, Comedy, Thriller]    100.0   
1496  [Action, Adventure, Comedy]    100.0   
1497       [Action, Crime, Drama]    155.0   
1498             [Action, Horror]     86.0   
1499  [Action, Adventure, Sci-Fi]     88.0   

                                                   cast  num_mflix_comments  \
0     [Pearl White, Crane Wilbur, Paul Panzer, Edwar...                   0   
1     [Harold Lloyd, Mildred Davis, 'Snub' Pollard, ...                   0   
2     [Ronald Colman, Neil Hamilton, Ralph Forbes, A...                   0   
3     [Billie Dove, Tempe Pigott, Donald Crisp, Sam ...                   1   
4     [Harold Lloyd, Jobyna Ralston, Noah Young, Jim...                   0   
...                                                 ...                 ...   
1495  [Zack Ward, Dave Foley, Chris Coppola, Jackie ...                   0   
1496  [Tim Allen, John Travolta, Martin Lawrence, Wi...                   0   
1497  [Ajay Devgn, Kareena Kapoor, Saif Ali Khan, Ko...                   1   
1498  [Mena Suvari, Nick Cannon, Michael Welch, Anna...                   1   
1499  [Hayden Christensen, Jamie Bell, Rachel Bilson...                   0   

                                                 poster  \
0     https://m.media-amazon.com/images/M/MV5BMzgxOD...   
1     https://m.media-amazon.com/images/M/MV5BNzE1OW...   
2                                                  None   
3     https://m.media-amazon.com/images/M/MV5BMzU0ND...   
4     https://m.media-amazon.com/images/M/MV5BMTcxMT...   
...                                                 ...   
1495  https://m.media-amazon.com/images/M/MV5BMTIzMD...   
1496  https://m.media-amazon.com/images/M/MV5BZWZlMz...   
1497  https://m.media-amazon.com/images/M/MV5BY2NmNj...   
1498  https://m.media-amazon.com/images/M/MV5BNzg1Mj...   
1499  https://m.media-amazon.com/images/M/MV5BMjEwOT...   

                      title  \
0     The Perils of Pauline   
1        From Hand to Mouth   
2                Beau Geste   
3          The Black Pirate   
4         For Heaven's Sake   
...                     ...   
1495                 Postal   
1496              Wild Hogs   
1497                 Omkara   
1498        Day of the Dead   
1499                 Jumper   

                                               fullplot  \
0     Young Pauline is left a lot of money when her ...   
1     As a penniless man worries about how he will m...   
2     Michael "Beau" Geste leaves England in disgrac...   
3     A nobleman vows to avenge the death of his fat...   
4     The Uptown Boy, J. Harold Manners (Lloyd) is a...   
...                                                 ...   
1495  The story begins with a regular Joe who tries ...   
1496  Four middle-aged men decide to take a road tri...   
1497  Advocate Raghunath Mishra has arranged the mar...   
1498  In Leadville, Colorado, Captain Rhodes and his...   
1499  David Rice is a high s

In [9]:
!pip install PyMuPDF transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 35.3 MB/s eta 0:00:00


In [10]:
import fitz
from transformers import pipeline
from datasets import load_dataset
import numpy as np

embedding_model = SentenceTransformer('all-MiniLM-L6-v2') #The mini LM For embedding
summarizer = pipeline('summarization',model="facebook/bart-large-cnn") #Using BART's summarization capabilities
dataset = load_dataset("cnn_dailymail", "3.0.0", split="test[:5%]") #Test for now, will be replaced with ours!

documents = [article["article"]for article in dataset]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

validation-00000-of-00001.parquet:   0%|          | 0.00/34.7M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [21]:
def embed_documents(documents):
  doc_embeddings = []
  doc_chunks = []
  for doc in documents:
    chunks = doc.split('. ')
    embeddings = embedding_model.encode(chunks) #The crux of the function, Here the model will be returning the embedded input

    doc_chunks.extend(chunks)
    doc_embeddings.extend(embeddings)

  return np.array(doc_embeddings), doc_chunks #This is questionable, I dont understand why embeddings is being converted to a np array.

doc_embeddings, doc_chunks = embed_documents(documents)

In [19]:
from sklearn.metrics.pairwise import cosine_similarity

def retrieve_relevant_chunk(query,doc_embeddings, doc_chunks,top_k = 5):
  query_embedding = embedding_model.encode(query).reshape(1,-1)
  similarities = cosine_similarity(query_embedding,doc_embeddings).flatten()
  top_k_indices = similarities.argsort()[-top_k:][::-1]

  return [doc_chunks[idx] for idx in top_k_indices]

In [13]:
def generate_summary(relevant_chunks):
  combined_texts = " ".join(relevant_chunks)
  summary = summarizer(combined_texts,max_length=150,min_length=30,do_sample=False)
  return summary

In [22]:
query = "Give a brief summary of the document."
relevant_chunks = retrieve_relevant_chunk(query, doc_embeddings, doc_chunks)
summary = generate_summary(relevant_chunks)

print("Summary:")
print(summary)


Your max_length is set to 150, but your input_length is only 81. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=40)


Summary:
[{'summary_text': 'Unclassified summary of that information will be provided "to the extent feasible," court papers say. Document includes ideas from Mook, along with a wide array of Clinton\'s soon-to-be staffers and advisers.'}]
